In [4]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict
from google.colab import drive
#drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [3]:
url_train_dev = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSZRYmir99b4BqU9UwkJvVIJRBCVgbMye_zDzHwvtjFhvQYJXk9Q3q-spYGse0B5kgVWbEDuxiineIn/pub?gid=1591461788&single=true&output=csv'

#url_test = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vT-KNR9nuYatLkSbzSRgpz6Ku1n4TN4w6kKmFLkA6QJHTfQzmX0puBsLF7PAAQJQAxUpgruDd_RRgK7/pub?gid=417546901&single=true&output=tsv'

# Load and inspect the data

In [4]:
from io import StringIO
import requests

def load_dataset(url):
    #r = requests.get(url)
    #data = r.content.decode('utf8')
    df = pd.read_csv(open(link,encoding="utf8"))
    df.columns = ['bug-id', 'bug', 'label']
    return df

In [5]:
link = "C:/Users/David/Documents/GitHub/Data_Science_for_Software_Engineering/Project/testing_list.csv"

df = pd.read_csv(open(link,encoding="utf8"))

df.head(100)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 7115: invalid start byte

In [6]:
df_train_dev = load_dataset(link)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x83 in position 7115: invalid start byte

In [36]:
print('Infos train-dev-set:')
print(df_train_dev.info())

Infos train-dev-set:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   bug-id  199 non-null    int64 
 1   bug     199 non-null    object
 2   label   199 non-null    object
dtypes: int64(1), object(2)
memory usage: 4.8+ KB
None


In [37]:
df_train_dev.head()

,bug-id,bug,label
0,104115,The browser_wa_properties-view-media-nodes.js ...,blocker
1,145099,Browser Toolbox can't be resized,blocker
2,59337,WebConsole helper function $ always returns null,blocker
3,70270,Web console is buggy in window mode,blocker
4,135639,browser_perf-button-states.js and browser_mark...,blocker


In [38]:
print(df_train_dev.label.unique())

['blocker']


In [39]:
df_train_dev.groupby('label').size().sort_values(ascending = False).plot.bar(figsize=(20, 5))

In [ ]:
df_train_dev.groupby('label').size().sort_values(ascending = False)

label
en         18508
ja         10421
es          5930
und         4537
id          3006
           ...  
dv             1
tn             1
ta_LATN        1
si             1
ja_LATN        1
Length: 69, dtype: int64

# Deal with class imbalance by unifying very rare classes (1 member) and upsampling rare classes

In [33]:
# df_train_dev['label'] = df_train_dev['label'].apply(lambda x: x.strip().lower())
# df_test['label'] = df_test['label'].apply(lambda x: x.strip().lower())

In [34]:
def unify_singleton_classes(df, threshold):
    counts = df['label'].value_counts()
    rare_labels = [label for label in counts[counts < threshold].index]
    df.loc[(df.label.isin(rare_labels)), 'label'] = 'und'
    return df

df_train_dev_uni = unify_singleton_classes(df_train_dev, 3)

NameError: ignored

In [31]:
from sklearn.utils import resample

def upsample_rare_classes(df, threshold, sample_size):
    """Copy low frequent Tweets of languages, that have a number 
    of tweets below a certain threshold.
    
    :param data_train: dataframe
    :param threshold: int
    :param sample_size: int
    """
    class_counts = defaultdict(int)
    classes_to_upsample = set()
    for index, row in df.iterrows():
        class_counts[row['label']] += 1
    for label, count in class_counts.items():
        if count < threshold:
            classes_to_upsample.add(label)
    df_to_upsample = df[df['label'].isin(classes_to_upsample)]
    upsampled_classes = pd.DataFrame(resample(df_to_upsample.to_numpy(), replace=True, n_samples=sample_size, random_state=42))
    upsampled_classes.columns = ['tweet', 'label']
    return pd.concat([df, upsampled_classes]).sample(frac=1)

df_train_dev_up = upsample_rare_classes(df=df_train_dev_uni, threshold=50, sample_size=2000)

NameError: ignored

In [ ]:
print(df_train_dev.shape)
print(df_train_dev_up.shape)

(52675, 2)
(54675, 2)


# Process labels

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_fitted = LabelEncoder().fit(df_train_dev_up['label'])

In [ ]:
# map all classes that are not in train_dev to undefined
for i, label in enumerate(df_test['label']):
    df_test['label'][i] = 'und' if label not in le_fitted.classes_ else label
# check if it worked: should return an empty list
print([label for label in df_test['label'] if label not in set(df_train_dev_up['label'])])

[]


In [ ]:
y_train_dev, y_test = le_fitted.transform(df_train_dev_up['label']), le_fitted.transform(df_test['label'])

# Preprocess Tweets

Pipeline classes:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


class TweetNormalizer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self 

    def _normalize_tweet(self, tweet):
        """Remove punctuation and newlines, lowercase, pad with spaces.

        :param tweet: string
        :return: normalized string
        """
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))
        tweet = re.sub(r'\n', r'', tweet)
        tweet = tweet.lower()
        tweet = re.sub(r'@\w+\b', r'', tweet)
        tweet = re.sub(r'\b\S+//\S+\b', r'', tweet)
        # tweet = ' ' + tweet + ' '
        return tweet
    
    def transform(self, X, y=None):
        tweets = []
        for tweet in X:
            tweets.append(self._normalize_tweet(tweet))
        return np.array(tweets)


class FeatureExtractor(BaseEstimator, TransformerMixin):


    vowels = set([c for c in 'aeiouäöüàéèëï'])
    consonants = set([c for c in 'bcdfghklmnlpqrstvwxyz'])

    def __init__(self):
        self.scaler = MinMaxScaler()

    def _to_bigrams(self, tweet):
        return [bg[0] + bg[1] for bg in zip(tweet, tweet[1:])]

    def _get_vowel_consonant_ratio(self, tweet):
        vf = 0
        cf = 0
        for c in tweet.lower():
            if c in self.vowels:
                vf =+ 1
            elif c in self.consonants:
                cf += 1
        return vf / (cf + 1)

    def _get_capitalization_ratio(self, tweet):
        up_count = 0
        for c in tweet:
            if c.upper() == c:
                up_count += 1
        return up_count / (len(tweet) + 1)

    def _get_double_char_freq(self, tweet):
        double_freq = 0
        for bg in self._to_bigrams(tweet):
            if bg[0] == bg[1]:
                double_freq += 1
        return double_freq
    
    def _extract_num_features(self, tweets):
        num_features = []
        for tweet in tweets:
            feat_tweet = []
            feat_tweet.append(self._get_vowel_consonant_ratio(tweet))
            feat_tweet.append(self._get_capitalization_ratio(tweet))
            feat_tweet.append(self._get_double_char_freq(tweet))
            num_features.append(feat_tweet)
        return np.array(num_features)
    
    def fit(self, X, y=None):
        numerical_features = self._extract_num_features(X)
        self.scaler.fit(numerical_features)
        return self
    
    def transform(self, X, y=None):
        numerical_features= self._extract_num_features(X)
        return X, self.scaler.transform(numerical_features)


class MatrixToArrayConverter1(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X[0].toarray(), X[1]


class MatrixUnifier(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return np.concatenate([X[0].todense(), X[1]], axis=1)


class CountVectorizerWrapper:

    def __init__(self, ngram_range, analyzer, max_features, binary):
        print('args:', str([ngram_range, analyzer, max_features, binary]))
        self.countvec = CountVectorizer(ngram_range=ngram_range, analyzer=analyzer, max_features=max_features, binary=binary)
    
    def fit(self, X, y=None):
        tweets, numerical_features = X
        self.countvec.fit(tweets)
        return self
    
    def transform(self, X, y=None):
        tweets, numerical_features = X
        return self.countvec.transform(tweets), numerical_features


class OneHotEncoderWrapper:

    def __init__(self, handle_unknown):
        self.ohe = OneHotEncoder(handle_unknown=handle_unknown)
    
    def fit(self, X, y=None):
        self.ohe.fit(X[0])
        return self
    
    def transform(self, X, y=None):
        return self.ohe.transform(X[0]), X[1]


Helper classes for the pipeline:

In [ ]:
class GenericClassifier(BaseEstimator):

    def __init__(self, estimator):
        self.clf = clf


    def fit(self, X, y=None, **kwargs):
        self.clf.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.clf.predict(X)


    def predict_proba(self, X):
        return self.clf.predict_proba(X)


    def score(self, X, y):
        return self.clf.score(X, y)


class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        import pdb; pdb.set_trace()
        return X

    def fit(self, X, y=None, **fit_params):
        return self

# GridSearch and Training

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier

In [ ]:
clf_param_grid = {
    'MultinomialNB': [MultinomialNB, {'CLF__alpha': [0.1, 1]}],
    'SGDClassifier': [SGDClassifier, {'CLF__loss': ['hinge', 'log'], 'CLF__penalty': ['l2', 'l1'], 'CLF__max_iter': [100, 300], 'CLF__early_stopping': [True, False]}]
}

In [ ]:
models = []
for clf_name in clf_param_grid:
    print(30*'-')
    print(clf_name)
    param_grid = clf_param_grid[clf_name][1]
    print(param_grid)
    bigram_vec_args = dict(ngram_range=(2,2), analyzer='char_wb', max_features=100, binary=True)
    pipe = Pipeline(steps=[
        ('TweetNormalizer', TweetNormalizer()),
        ('FeatureExtractor', FeatureExtractor()),
        ('BigramVectorizer', CountVectorizerWrapper(**bigram_vec_args)),
        ('MatrixToArrayConverter', MatrixToArrayConverter1()),
        ('OneHotEncoder', OneHotEncoderWrapper(handle_unknown='ignore')),
        ('MatrixUnifier', MatrixUnifier()),
        ('CLF', clf_param_grid[clf_name][0]())
    ], verbose=True)
    grid = GridSearchCV(pipe, n_jobs=1, param_grid=param_grid, scoring='f1_micro', cv=10)
    grid.fit(df_train_dev_up['tweet'].to_numpy(), y_train_dev)
    models.append(grid)

------------------------------
MultinomialNB
{'CLF__alpha': [0.1, 1]}
args: [(2, 2), 'char_wb', 100, True]
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   2.4s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.6s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   7.5s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.6s
[Pipeline] ..... (step 6 of 7) Processing MatrixUnifier, total=   0.1s
[Pipeline] ............... (step 7 of 7) Processing CLF, total=   0.1s
[Pipeline] ... (step 1 of 7) Processing TweetNormalizer, total=   1.2s
[Pipeline] .. (step 2 of 7) Processing FeatureExtractor, total=   5.7s
[Pipeline] .. (step 3 of 7) Processing BigramVectorizer, total=   7.6s
[Pipeline]  (step 4 of 7) Processing MatrixToArrayConverter, total=   0.0s
[Pipeline] ..... (step 5 of 7) Processing OneHotEncoder, total=   0.6s
[Pipeline] ..... (step 6 of 7) Pr

# Results

Micro f1-Score of the naive base models on the dev set:

In [ ]:
models[0].cv_results_

{'mean_fit_time': array([15.36567056, 14.83670509]),
 'mean_score_time': array([0.76010673, 0.68476903]),
 'mean_test_score': array([0.61991759, 0.61781428]),
 'param_CLF__alpha': masked_array(data=[0.1, 1],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'CLF__alpha': 0.1}, {'CLF__alpha': 1}],
 'rank_test_score': array([1, 2], dtype=int32),
 'split0_test_score': array([0.60442575, 0.60259693]),
 'split1_test_score': array([0.62746891, 0.62692026]),
 'split2_test_score': array([0.62070227, 0.61795903]),
 'split3_test_score': array([0.61942209, 0.61558157]),
 'split4_test_score': array([0.63332114, 0.63039503]),
 'split5_test_score': array([0.62301079, 0.6184379 ]),
 'split6_test_score': array([0.62173038, 0.6208158 ]),
 'split7_test_score': array([0.6067313 , 0.60837754]),
 'split8_test_score': array([0.62118164, 0.61916956]),
 'split9_test_score': array([0.62118164, 0.61788915]),
 'std_fit_time': array([1.20570028, 0.84751295]),
 'st

Micro and macro f1-score of the best naive bayes model on the test set:

In [ ]:
from sklearn.metrics import f1_score
preds = models[0].predict(df_test['tweet'].to_numpy())
f1_micro = f1_score(preds, y_test, average='micro')
f1_macro = f1_score(preds, y_test, average='macro')
print(f'F1-micro-score on the testset: {f1_micro}')
print(f'F1-macro-score on the testset: {f1_macro}')

F1-micro-score on the testset: 0.6298667068303336
F1-macro-score on the testset: 0.1891293449074721


Micro f1-Score of the SGD models on the dev set:

In [ ]:
models[1].cv_results_

{'mean_fit_time': array([34.92653723, 50.45851586, 34.84295452, 50.16186893, 42.44045622,
        60.79770355, 43.09442978, 60.35572217, 40.88421643, 77.39089243,
        40.50925915, 77.24433172, 44.00839155, 97.71396422, 44.11117506,
        99.27012889]),
 'mean_score_time': array([0.69382052, 0.6921206 , 0.69145575, 0.68961115, 0.69149637,
        0.68872337, 0.69514825, 0.69482107, 0.64930549, 0.64660978,
        0.64958162, 0.64531262, 0.64991169, 0.65115962, 0.68658216,
        0.64630191]),
 'mean_test_score': array([0.66738063, 0.66610009, 0.6451774 , 0.68347514, 0.68642086,
        0.69964359, 0.6872975 , 0.69651563, 0.72418882, 0.71564712,
        0.71487943, 0.71253834, 0.74015567, 0.73256524, 0.74222244,
        0.73093762]),
 'param_CLF__early_stopping': masked_array(data=[True, True, True, True, True, True, True, True, False,
                    False, False, False, False, False, False, False],
              mask=[False, False, False, False, False, False, False, False,
 

Accuracy of the best SGD model on the test set:

In [ ]:
preds = models[1].predict(df_test['tweet'].to_numpy())
f1_micro = f1_score(preds, y_test, average='micro')
f1_macro = f1_score(preds, y_test, average='macro')
print(f'F1-micro-score on the testset: {f1_micro}')
print(f'F1-macro-score on the testset: {f1_macro}')

F1-micro-score on the testset: 0.768054823405377
F1-macro-score on the testset: 0.23793764312477245


Let's check the confusion matrix:

In [ ]:
num_classes = len(le_fitted.classes_)
def create_confusion_matrix(num_classes, preds, y_test):
    """Create confusion matrix 'by hand' since test set does not contain all labels (thanks to Sarah Kiener)."""
    df = pd.DataFrame(np.zeros((num_classes, num_classes), dtype=int))
    for i, j in zip(preds, y_test):
        df.iloc[i, j] += 1
    df.columns = le_fitted.classes_
    df.index = le_fitted.classes_
    return df
df = create_confusion_matrix(num_classes, preds, y_test)
df

,ar,ar_LATN,bn,bs,ca,cs,da,de,el,en,es,fa,fi,fr,gl,he,hi,hi-Latn,hr,hu,id,it,ja,jv,ko,ms,ne,nl,no,pl,pt,ro,ru,sq,sr,su,sv,sw,ta,th,tl,tr,uk,und,ur,ur_LATN,vi,zh-CN,zh-TW
ar,203,1,0,0,0,0,0,0,0,5,0,0,0,0,0,2,0,0,0,0,2,0,227,0,12,0,0,0,0,0,0,0,2,0,0,0,0,0,1,10,0,0,0,11,1,0,0,0,3
ar_LATN,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
bn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
bs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
ca,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
cs,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
da,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
de,0,0,0,0,0,0,0,27,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
el,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
en,3,2,0,0,1,0,1,15,1,4403,60,0,0,52,0,1,0,3,0,0,66,7,34,1,3,6,0,21,1,8,48,0,25,0,1,0,5,0,0,2,19,17,0,238,1,1,1,0,0
